In [1]:
import pandas as pd
import numpy as np
from src.static import DATA_DIR

In [2]:
mapped_data = pd.read_csv(f'{DATA_DIR}/mapped_data.csv', na_values=['.','NaN', 'None'])
mapped_data.sort_values(by=['year', 'enterprise_flag', 'record_number'], inplace=True)
mapped_data = mapped_data[mapped_data.year >= (2023-5)]

In [3]:
groups = ['year', 'enterprise_flag', 'record_number', '']
test = mapped_data.groupby(['year', 'enterprise_flag', 'record_number']).agg(set).reset_index()
test.head()

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_bedrooms,num_units,affordability_level,tenant_income_ind
0,2018,fannie,1,{<10%},"{>10, <=120%}","{>=20%, >=40%}",{prior to year aquired},{refinance},{mortgage_company},{no},{25-50},"{>=2, 0-1}",{1.0},"{>=0, <=50%}",{No}
1,2018,fannie,2,{>=30% <100%},"{>0, <=80%}","{<20%, >=40%}",{prior to year aquired},{purchase},{mortgage_company},{no},{25-50},{>=2},{1.0},"{>50, <=60%, >=0, <=50%, >60, <=80%}",{No}
2,2018,fannie,3,{>=30% <100%},"{>0, <=80%}","{<20%, <40%}",{prior to year aquired},{refinance},{mortgage_company},{no},{25-50},{>=2},{1.0},"{>80, <=100%, >60, <=80%}",{No}
3,2018,fannie,4,{>=30% <100%},"{>0, <=80%}","{<20%, <40%}",{prior to year aquired},{refinance},{mortgage_company},{no},{25-50},{0-1},{1.0},"{>50, <=60%, >=0, <=50%, >60, <=80%, >80, <=100%}",{No}
4,2018,fannie,5,{>=30% <100%},"{>0, <=80%}","{>=20%, >=40%}",{prior to year aquired},{purchase},{mortgage_company},{no},{5 - 24},{>=2},{1.0},"{>50, <=60%, >=0, <=50%}",{No}


In [6]:
df = pd.get_dummies(
    mapped_data,
      columns=['num_bedrooms', 'affordability_level']
    )

In [12]:
df.columns = df.columns.str.strip()
df.columns

Index(['year', 'enterprise_flag', 'record_number', 'census_tract_2020',
       'tract_income_ratio', 'affordability_cat', 'date_of_mortgage_note',
       'purpose_of_loan', 'type_of_seller', 'federal_guarantee',
       'tot_num_units', 'num_units', 'tenant_income_ind', 'num_bedrooms_0-1',
       'num_bedrooms_>=2', 'affordability_level_>100%',
       'affordability_level_>50, <=60%', 'affordability_level_>60, <=80%',
       'affordability_level_>80, <=100%', 'affordability_level_>=0, <=50%'],
      dtype='object')

In [13]:
def unit_count_transformer(df, cols):
    for col in cols:
        df[col] = df.index.map(lambda x: df.loc[x]['num_units'] if df[col].loc[x] else 0)
    return df

In [14]:
unit_count_transformer(df, ['num_bedrooms_0-1', 'num_bedrooms_>=2', 'affordability_level_>100%',
                     'affordability_level_>50, <=60%', 'affordability_level_>60, <=80%', 
                     'affordability_level_>80, <=100%', 'affordability_level_>=0, <=50%']);

In [ ]:
df = df.groupby(
    ['year', 'enterprise_flag', 'record_number', 'census_tract_2020', 'tract_income_ratio',
     'date_of_mortgage_note', 'purpose_of_loan', 'type_of_seller', 'federal_guarantee',
     'tenant_income_ind', 'affordability_cat', 'tot_num_units']
    )[['num_units', 'num_bedrooms_0-1', 'num_bedrooms_>=2', 'affordability_level_>100%',
       'affordability_level_>50, <=60%', 'affordability_level_>60, <=80%',
       'affordability_level_>80, <=100%', 'affordability_level_>=0, <=50%']].agg('sum').reset_index()

In [22]:
df.head()

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_units,tenant_income_ind,num_bedrooms_0-1,num_bedrooms_>=2,affordability_level_>100%,"affordability_level_>50, <=60%","affordability_level_>60, <=80%","affordability_level_>80, <=100%","affordability_level_>=0, <=50%"
748416,2018,fannie,1,<10%,">10, <=120%",">=20%, >=40%",prior to year aquired,refinance,mortgage_company,no,25-50,1.0,No,1.0,0.0,0.0,0.0,0.0,0.0,1.0
748417,2018,fannie,1,<10%,">10, <=120%",">=20%, >=40%",prior to year aquired,refinance,mortgage_company,no,25-50,1.0,No,0.0,1.0,0.0,0.0,0.0,0.0,1.0
748418,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,25-50,1.0,No,0.0,1.0,0.0,1.0,0.0,0.0,0.0
748419,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,25-50,1.0,No,0.0,1.0,0.0,0.0,1.0,0.0,0.0
748420,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,25-50,1.0,No,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
df = pd.get_dummies(df, columns=[])

num_units  \
year enterprise_flag record_number census_tract_2020 tract_income_ratio date_of_mortgage_note purpose_of_loan type_of_seller   federal_guarantee tenant_income_ind affordability_cat tot_num_units              
2018 fannie          1             <10%              >10, <=120%        prior to year aquired refinance       mortgage_company no                No                >=20%, >=40%      25-50                2.0   
                     2             >=30% <100%       >0, <=80%          prior to year aquired purchase        mortgage_company no                No                <20%, >=40%       25-50                3.0   
                     3             >=30% <100%       >0, <=80%          prior to year aquired refinance       mortgage_company no                No                <20%, <40%        25-50                2.0   
                     4             >=30% <100%       >0, <=80%          prior to year aquired refinance       mortgage_company no                No                <20%, <40%        25-50                4.0   
                     5             >=30% <100%       >0, <=80%          prior to year aquired purchase        mortgage_company no                No                >=20%, >=40%      5 - 24               2.0   

                                                                                                                                                                                                    num_bedrooms_0-1  \
year enterprise_flag record_number census_tract_2020 tract_income_ratio date_of_mortgage_note purpose_of_loan type_of_seller   federal_guarantee tenant_income_ind affordability_cat tot_num_units                     
2018 fannie          1             <10%              >10, <=120%        prior to year aquired refinance       mortgage_company no                No                >=20%, >=40%      25-50                       1.0   
                     2             >=30% <100%       >0, <=80%          prior to year aquired purchase        mortgage_company no                No                <20%, >=40%       25-50                       0.0   
                     3             >=30% <100%       >0, <=80%          prior to year aquired refinance       mortgage_company no                No                <20%, <40%        25-50                       0.0   
                     4             >=30% <100%       >0, <=80%          prior to year aquired refinance       mortgage_company no                No                <20%, <40%        25-50                       4.0   
                     5             >=30% <100%       >0, <=80%          prior to year aquired purchase        mortgage_company no                No                >=20%, >=40%      5 - 24                      0.0   

                                                                                                                                                                                                    num_bedrooms_>=2  \
year enterprise_flag record_number census_tract_2020 tract_income_ratio date_of_mortgage_note purpose_of_loan type_of_seller   federal_guarantee tenant_income_ind affordability_cat tot_num_units                     
2018 fannie          1             <10%              >10, <=120%        prior to year aquired refinance       mortgage_company no                No                >=20%, >=40%      25-50                       1.0   
                     2             >=30% <100%       >0, <=80%          prior to year aquired purchase        mortgage_company no                No                <20%, >=40%       25-50                       3.0   
                     3             >=30% <100%       >0, <=80%          prior to year aquired refinance       mortgage_company no                No                <20%, <40%        25-50                       2.0   
                     4             >=30% <100%       >0, <=80%          prior to year aquired refinance       mortgage_company no          

In [4]:
agg_cols = ['census_tract_2020', 'tract_income_ratio', 'affordability_cat', 'date_of_mortgage_note',
             'purpose_of_loan', 'type_of_seller', 'federal_guarantee', 'tot_num_units',
               'num_bedrooms', 'num_units', 'affordability_level', 'tenant_income_ind']
counts = test[agg_cols].map(len)
counts.head()

,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,tot_num_units,num_bedrooms,num_units,affordability_level,tenant_income_ind
0,1,1,1,1,1,1,1,1,2,1,1,1
1,1,1,1,1,1,1,1,1,1,1,3,1
2,1,1,1,1,1,1,1,1,1,1,2,1
3,1,1,1,1,1,1,1,1,1,1,4,1
4,1,1,1,1,1,1,1,1,1,1,2,1


In [5]:
add_to_gb = []
for col in counts:
    print(f'{col} ', counts[col].unique())
    if len(counts[col].unique()) == 1:
        add_to_gb.append(col)

census_tract_2020  [1]
tract_income_ratio  [1]
affordability_cat  [1]
date_of_mortgage_note  [1]
purpose_of_loan  [1]
type_of_seller  [1]
federal_guarantee  [1]
tot_num_units  [1]
num_bedrooms  [2 1]
num_units  [ 1  6  7  4 10  5 11 13  2  9 16 12  3  8 17 18 15 14 21 19 25 31 20 22
 23 27 39 24 35 34 29 28 33]
affordability_level  [1 3 2 4 5 6]
tenant_income_ind  [1]


In [16]:
ohe_data = pd.get_dummies(mapped_data, columns=['tot_num_units', 'num_bedrooms', 'affordability_level',])
ohe_data.columns = ohe_data.columns.str.strip().str.replace(' - ', '-')
ohe_data.head()

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,...,tot_num_units_5-24,tot_num_units_51-99,tot_num_units_> 149,num_bedrooms_0-1,num_bedrooms_>=2,affordability_level_>100%,"affordability_level_>50, <=60%","affordability_level_>60, <=80%","affordability_level_>80, <=100%","affordability_level_>=0, <=50%"
748416,2018,fannie,1,<10%,">10, <=120%",">=20%, >=40%",prior to year aquired,refinance,mortgage_company,no,...,False,False,False,True,False,False,False,False,False,True
748417,2018,fannie,1,<10%,">10, <=120%",">=20%, >=40%",prior to year aquired,refinance,mortgage_company,no,...,False,False,False,False,True,False,False,False,False,True
748418,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,...,False,False,False,False,True,False,True,False,False,False
748419,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,...,False,False,False,False,True,False,False,True,False,False
748420,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,...,False,False,False,False,True,False,False,False,False,True


In [18]:
multi_value_cols = ['num_bedrooms_0-1', 'num_bedrooms_>=2', 'affordability_level_>100%',
                     'affordability_level_>50, <=60%', 'affordability_level_>60, <=80%', 
                     'affordability_level_>80, <=100%', 'affordability_level_>=0, <=50%']

ohe_data = unit_count_transformer(
    ohe_data,
    multi_value_cols
)

In [19]:
ohe_data.head()

,year,enterprise_flag,record_number,census_tract_2020,tract_income_ratio,affordability_cat,date_of_mortgage_note,purpose_of_loan,type_of_seller,federal_guarantee,...,tot_num_units_5-24,tot_num_units_51-99,tot_num_units_> 149,num_bedrooms_0-1,num_bedrooms_>=2,affordability_level_>100%,"affordability_level_>50, <=60%","affordability_level_>60, <=80%","affordability_level_>80, <=100%","affordability_level_>=0, <=50%"
748416,2018,fannie,1,<10%,">10, <=120%",">=20%, >=40%",prior to year aquired,refinance,mortgage_company,no,...,False,False,False,1.0,0.0,0.0,0.0,0.0,0.0,1.0
748417,2018,fannie,1,<10%,">10, <=120%",">=20%, >=40%",prior to year aquired,refinance,mortgage_company,no,...,False,False,False,0.0,1.0,0.0,0.0,0.0,0.0,1.0
748418,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,...,False,False,False,0.0,1.0,0.0,1.0,0.0,0.0,0.0
748419,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,...,False,False,False,0.0,1.0,0.0,0.0,1.0,0.0,0.0
748420,2018,fannie,2,>=30% <100%,">0, <=80%","<20%, >=40%",prior to year aquired,purchase,mortgage_company,no,...,False,False,False,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [20]:
gbcols = list(set(ohe_data.columns) - set(multi_value_cols))
gbcols

['year',
 'tot_num_units_5-24',
 'tract_income_ratio',
 'type_of_seller',
 'census_tract_2020',
 'tot_num_units_51-99',
 'tot_num_units_25-50',
 'enterprise_flag',
 'tenant_income_ind',
 'date_of_mortgage_note',
 'tot_num_units_100-149',
 'affordability_cat',
 'record_number',
 'federal_guarantee',
 'num_units',
 'purpose_of_loan',
 'tot_num_units_> 149']

In [22]:
ohe_data.head()

,year,tot_num_units_5-24,tract_income_ratio,type_of_seller,census_tract_2020,tot_num_units_51-99,tot_num_units_25-50,enterprise_flag,tenant_income_ind,date_of_mortgage_note,...,num_units,purpose_of_loan,tot_num_units_> 149,num_bedrooms_0-1,num_bedrooms_>=2,affordability_level_>100%,"affordability_level_>50, <=60%","affordability_level_>60, <=80%","affordability_level_>80, <=100%","affordability_level_>=0, <=50%"
0,2018,False,9,Other,>=30% <100%,False,False,freddie,No,same year as acquired,...,1.0,refinance,False,3.0,0.0,1.0,0.0,1.0,1.0,0.0
1,2018,False,9,Other,>=30% <100%,False,True,fannie,No,same year as acquired,...,1.0,refinance,False,1.0,2.0,0.0,1.0,0.0,0.0,2.0
2,2018,False,9,Other,>=30% <100%,False,True,freddie,No,same year as acquired,...,1.0,refinance,False,3.0,1.0,0.0,1.0,2.0,0.0,1.0
3,2018,False,9,Other,>=30% <100%,False,True,freddie,No,same year as acquired,...,2.0,refinance,False,6.0,0.0,0.0,2.0,2.0,0.0,2.0
4,2018,False,9,Other,>=30% <100%,False,True,freddie,No,same year as acquired,...,3.0,refinance,False,3.0,0.0,0.0,3.0,0.0,0.0,0.0


In [21]:
ohe_data = ohe_data.groupby(gbcols).agg('sum').reset_index()

In [ ]:
ohe_data['after_covid_ind'] = ohe_data.year > 2020
ohe_data.head()